### 문제

아래 조건을 만족하며 turtlesim 패키를 제어하는 노드 코드를 구현하시오.

- 제어 조건
    
    1. 첫 번째 거북이는 터미널에 사용자가 입력한 위치로 이동
    2. 거북이 속도는 0.5로 설정하며, 노드 내 토픽 메시지는 10Hz로 발행
    

### 반환값

- 시작점에서 도착점까지의 최단 경로를 나타내는 좌표 리스트 [(x1, y1), (x2, y2), ...]

### 참고

```python
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist
from turtlesim.msg import Pose as TurtlePose
import math
```

- **`rospy`**를 사용하여 노드를 생성하고, **`geometry_msgs/Twist`** 메시지를 통해 거북이를 제어


In [ ]:
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist
from turtlesim.msg import Pose as TurtlePose
import math

class TurtleControlNode(Node):
    def __init__(self):
        super().__init__('turtle_control_node')
        self.publisher_ = self.create_publisher(Twist, 'turtle1/cmd_vel', 10)
        self.subscriber_ = self.create_subscription(TurtlePose, 'turtle1/pose', self.pose_callback, 10)
        self.get_logger().info("노드 초기화 완료")
        self.current_turtle_pose = TurtlePose()
        self.target_pose = TurtlePose()
        self.is_moving = False

    def pose_callback(self, msg):
        self.current_turtle_pose = msg

        distance_to_target = self.calculate_distance(self.target_pose.x, self.target_pose.y)
        if distance_to_target < 0.1:
            self.get_logger().info("목표 지점에 도착하여 노드를 종료합니다.")
            self.is_moving = False
            self.destroy_node()

    def calculate_distance(self, target_x, target_y):
        return math.sqrt((target_x - self.current_turtle_pose.x)**2 + (target_y - self.current_turtle_pose.y)**2)

    def get_user_input(self):
        x = float(input("목표 x 좌표를 입력하세요: "))
        y = float(input("목표 y 좌표를 입력하세요: "))
        return x, y

    def run(self):
        self.get_logger().info("터틀 제어 노드를 시작합니다.")
        self.is_moving = True

        self.target_pose.x, self.target_pose.y = self.get_user_input()

        while rclpy.ok() and self.is_moving:
            msg = Twist()
            angle_to_target = math.atan2(self.target_pose.y - self.current_turtle_pose.y,
                                         self.target_pose.x - self.current_turtle_pose.x)
            msg.linear.x = 0.5
            msg.angular.z = angle_to_target - self.current_turtle_pose.theta

            self.publisher_.publish(msg)
            rclpy.spin_once(self)

        self.get_logger().info("터틀 제어 노드를 종료합니다.")

def main(args=None):
    rclpy.init(args=args)
    node = TurtleControlNode()
    node.run()
    rclpy.shutdown()

if __name__ == '__main__':
    main()
